In [1]:
import pandas as pd

In [ ]:
table_url = 'https://www.mapsofworld.com/thematic-maps/types-of-governments.html'
tables = pd.read_html(table_url)
tables

In [ ]:
gov_df = tables[5]
gov_df.columns = ['Country', 'Govt Type', 'Head of State', 'Head of Government']
new_gov_df = gov_df.rename(columns={'Country':'country','Govt Type':'govt_type', 'Head of State':'head_of_state', 'Head of Government':'head_of_government'})
new_gov_df.set_index('country', inplace=True)
new_gov_df

In [ ]:
from sqlalchemy import create_engine
from config import postgres_pwd

db_name = "happiness_db"
table_name = "countries_by_government"

connection_string = f"postgres:{postgres_pwd}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')
conn = engine.connect()

new_gov_df.to_sql(table_name, con=engine, if_exists="append", index=False)

conn.close()